In [1]:
import pandas as pd
from datetime import datetime

In [2]:
# Ruta del archivo .xlsx
file_path = "revision_documentos.xlsx"
file_path_destino = "revision_documentos_" + datetime.now().strftime("%Y%m%d%H%M%S") + ".xlsx"
# Lee el archivo Excel
df = pd.read_excel(file_path)

# Muestra las primeras filas del DataFrame
print(df.head())

      FECHA_REGISTRO_HH       FECHA_REGISTRO  ORIGENDOCID  \
0  8/12/2024 2:53:54 PM  2024-12-08 00:00:00            1   
1  8/12/2024 2:51:31 PM  2024-12-08 00:00:00            1   
2  8/12/2024 2:50:08 PM  2024-12-08 00:00:00            2   
3  8/12/2024 2:48:42 PM  2024-12-08 00:00:00            1   
4  8/12/2024 2:50:08 PM  2024-12-08 00:00:00            2   

                    ORIGEN  DOCO_ID  DOCO_TIDO_ID  ID_ESTADO   ESTADO  \
0  Documentos Contratistas  5652963             8         11  Enviado   
1  Documentos Contratistas  5652951             4         11  Enviado   
2       Documentos Riesgos  5652947            54         11  Enviado   
3  Documentos Contratistas  5652942             1         11  Enviado   
4       Documentos Riesgos  5652938            28         11  Enviado   

   DOCUMENTOSITUACIONID        DOCUMENTOSITUCION  ... Hora extendida  \
0                     2  Documento En renovacion  ...       14:53:54   
1                     2  Documento En renovacion  

In [3]:
# Para el primer analisis, se eliminan las columnas que no son relevantes
# se eliminan las columnas: DOCUMENTOVIGENCIA, FECHA_INICIO_DOCUMENTO, FECHA_FIN_DOCUMENTO
# DOCO_EMPR_RUT, TIPO, 'Hora extendida', 'Fecha y hora', 'Fecha.1', 'Gestionado','Tiempo desde ingreso', 'Tiempo para cumplir en plazo', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43' 'Unnamed: 29','Unnamed: 30', 'Fecha', 'Hora', 'AM/PM'
columnas_no_tenidas_en_cuenta = [
    'DOCUMENTOVIGENCIA', 'FECHA_INICIO_DOCUMENTO', 'FECHA_FIN_DOCUMENTO', 'DOCO_EMPR_RUT',
    'TIPO', 'Hora extendida', 'Fecha y hora', 'Fecha.1', 'Gestionado',
     'Tiempo desde ingreso', 'Tiempo para cumplir en plazo', 'Unnamed: 40',
     'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 29','Unnamed: 30',
     'Fecha', 'Hora', 'AM/PM']

df.drop(columnas_no_tenidas_en_cuenta,
         axis=1, inplace=True)

#verificamos la dimension del dataframe
print(df.shape)

(55587, 24)


In [4]:
# Creamos la columna tipo, en base a la columna estado. 
# Si el estado es "Enviado", el tipo es de "Entrada"
# Si el estado es "Rechazado" o "Validado", el tipo es de "Salida"

df["TIPO"] = df["ESTADO"].apply(lambda x: "Entrada" if x == "Enviado" else "Salida")

In [5]:
# Convertimos la columna FECHA_REGISTRO_HH, en formato mm/dd/yyyy hh:mm:ss PM/AM a datetime
df["FECHA_REGISTRO_HH"] = pd.to_datetime(df["FECHA_REGISTRO_HH"], format="%m/%d/%Y %I:%M:%S %p")

In [6]:
#creamos dos dataframes, uno con los documentos de entrada y otro con los de salida
df_entrada = df[df["TIPO"] == "Entrada"]
df_salida = df[df["TIPO"] == "Salida"]

In [7]:
print(df_entrada.shape)
print(df_salida.shape)

columnas_eliminadas_salida = ['FECHA_REGISTRO', 'ORIGENDOCID', 'ORIGEN',
       'DOCO_TIDO_ID', 'ID_ESTADO',
       'DOCUMENTOSITUACIONID', 'DOCUMENTOSITUCION', 'USUARIOREGISTRO',
       'USUARIOREGISTROCUENTA', 'USUARIOREGISTRONOMBRE',
       'DOCO_TIPO_VALIDACION', 'DOCO_CONT_ID', 'FECHA_INGRESO_DOCUMENTO'
       , 'DOCUMENTOESPECIALIDADID', 'DOCUMENTOESPECIALIDAD',
       'DOCUMENTOOCUPACIONID', 'DOCUMENTOOCUPACION',
       'DOCUMENTOCERTOCUPACIONID', 'DOCUMENTOCERTOCUPACION', 'TIPO']

(28869, 25)
(26718, 25)


In [8]:
# del dataframe de salidas, nos interesa conservar las columnas
# FECHA_REGISTRO_HH, ESTADO, USUARIOREGISTROID, pero renombramos a 
# FECHA_SALIDA, ESTADO_SALIDA, USUARIOID_SALIDA  para luego poder hacer el merge
df_salida = df_salida.rename(columns={"FECHA_REGISTRO_HH": "FECHA_SALIDA", "ESTADO": "ESTADO_SALIDA", "USUARIOREGISTROID": "USUARIOID_SALIDA"})
df_salida.drop(
       columnas_eliminadas_salida
       , axis=1, inplace=True)


In [9]:
# hacemos el merge, en base al doco_id, lo hacemos de tipo left para conservar los documentos de entrada
df_merged = pd.merge(df_entrada, df_salida, on="DOCO_ID", how="left")


El código de abajo es para ver si hay documentos que tienen salida pero no entrada, que en teoría no debería ocurrir
Una de las causas puede ser que la entrada este en otro documento

In [10]:
# Realizamos el merge de tipo right para conservar todos los documentos de salida
df_right_merged = pd.merge(df_entrada, df_salida, on="DOCO_ID", how="right")

# Filtramos los registros donde ID_ESTADO de entrada sea NaN
df_salida_sin_entrada = df_right_merged[df_right_merged['ID_ESTADO'].isna()]

# Mostramos los documentos de salida que no tienen un documento de entrada correspondiente
print(df_salida_sin_entrada)

      FECHA_REGISTRO_HH FECHA_REGISTRO  ORIGENDOCID ORIGEN  DOCO_ID  \
122                 NaT            NaN          NaN    NaN  5651209   
19419               NaT            NaN          NaN    NaN  5594599   
20479               NaT            NaN          NaN    NaN  5591897   
21594               NaT            NaN          NaN    NaN  5589216   
21761               NaT            NaN          NaN    NaN  5588854   
...                 ...            ...          ...    ...      ...   
26693               NaT            NaN          NaN    NaN   892984   
26694               NaT            NaN          NaN    NaN   889636   
26695               NaT            NaN          NaN    NaN   777211   
26696               NaT            NaN          NaN    NaN   732394   
26711               NaT            NaN          NaN    NaN   345448   

       DOCO_TIDO_ID  ID_ESTADO ESTADO  DOCUMENTOSITUACIONID DOCUMENTOSITUCION  \
122             NaN        NaN    NaN                   NaN       

In [11]:
#Agregamos una columna con la diferencia de tiempo entre entrada
# y salida de los documentos en horas
df_merged["DIFERENCIA_HORAS"] = (df_merged["FECHA_SALIDA"] - df_merged["FECHA_REGISTRO_HH"]).dt.total_seconds() / 3600

In [12]:
# Definimos el umbral de 48 horas
umbral = 48

# Agregamos un campo de desviación respecto al umbral
df_merged["DESVIACION"] = df_merged["DIFERENCIA_HORAS"] - umbral

# Determinamos el estado, teniendo en cuenta la desviación y los documentos
# sin salida

df_merged['estado'] = df_merged.apply(
    lambda row: 'Sin gestionar' if pd.isna(row['DIFERENCIA_HORAS']) 
    else ('Fuera de plazo' if row['DESVIACION'] > 0 
          else ('En plazo' if row['DESVIACION'] < 0 
                else 'Igual')), 
    axis=1
) 


In [13]:
# Obtener el mes actual y los tres meses anteriores
hoy = datetime.now()
mes_actual = hoy.month
año_actual = hoy.year

# Crear una lista de los tres meses anteriores y el mes actual
meses_validos = [(mes_actual - i - 1) % 12 + 1 for i in range(4)]
año_valido = año_actual if mes_actual in meses_validos else año_actual - 1

# Función para obtener la etiqueta del mes
def etiqueta_mes(fecha):
    mes = fecha.month
    año = fecha.year
    
    if mes == mes_actual and año == año_actual:
        return f"{hoy.strftime('%B')} {año_actual}"
    
    if mes in meses_validos:
        return f"{fecha.strftime('%B')} {año}"
    
    return 'Fuera de rango'

# Aplicar la función al DataFrame
df_merged['etiqueta_mes'] = df_merged['FECHA_REGISTRO_HH'].apply(etiqueta_mes)

print(df_merged)

        FECHA_REGISTRO_HH       FECHA_REGISTRO  ORIGENDOCID  \
0     2024-08-12 14:53:54  2024-12-08 00:00:00            1   
1     2024-08-12 14:51:31  2024-12-08 00:00:00            1   
2     2024-08-12 14:50:08  2024-12-08 00:00:00            2   
3     2024-08-12 14:48:42  2024-12-08 00:00:00            1   
4     2024-08-12 14:50:08  2024-12-08 00:00:00            2   
...                   ...                  ...          ...   
28954 2024-07-26 09:58:42            7/26/2024            4   
28955 2024-07-26 09:58:42            7/26/2024            4   
28956 2024-07-26 09:58:42            7/26/2024            4   
28957 2024-07-26 09:58:42            7/26/2024            4   
28958 2024-07-26 09:58:42            7/26/2024            4   

                               ORIGEN  DOCO_ID  DOCO_TIDO_ID  ID_ESTADO  \
0             Documentos Contratistas  5652963             8         11   
1             Documentos Contratistas  5652951             4         11   
2                 

In [14]:
df_merged.to_excel(file_path_destino, index=False)